Space to scratch out the mechanism for running all the processes I want to run on individual ROIs of classified cone mosaic data.

User Inputs (this will eventually be a .yml that the user edits and is read in by the single-mosaic-process executable)

In [15]:
# Paths
# -----
# load and save folder. all coordinate and image data must be in the load folder ()
# if there is already analyses for a mosaic that you want to add analyses to, it must 
# be located in the save file the same name as the coordinate file but with the save file-type
data_path = 'C:\\Users\\schle\Documents\\GitHub\\Incubator-2022-Geometry-of-Color\\CSV data all OCT locs Jan 2022\\'
save_path = 'Z:\\Sierra\\Incubator Data Pass\\'

# settings that define the files that will be searched for in the load folder above.
# processes indicated below will be run on all such files found. 
subject = ['AO001R', 'AO008R']
angle = ['temporal', 'nasal', 'superior', 'inferior']
eccentricity = ['1.5','3.5', '4', '4.5', '10']
conetype = ['all','L','M','S','unclassed']
conetype_color = ['y','r','g','b','w']
coord_unit = 'pixels' # units of the coordinate data loaded in - any string you want

# Run settings
# ------------
# select single-mosaic processes to run
# options: 'basic stats', 'intracone_dists', 'monteCarlo_Uniform', 'monteCarlo_coneLocked
# proc_to_run = {
#      'intracone_dist', 
# #     'monteCarlo_uniform', 
#      'monteCarlo_uniform_intracone_dist', 
# #     'monteCarlo_coneLocked',
#      'monteCarlo_coneLocked_intracone_dist',
#      'norm_by_MCU_mean'}
proc_to_run = {'norm_by_MCU_mean'}
    
# indicate whether all processes requested will be run on all mosaics found fitting the inputs above,
# or processes will only be run for a file if that process' outputs are not present in an identified save file
data_to_run = 'all' # 'all', absent_only

# Process-specific Settings
# -------------------------
# intracone distance settings
dist_area_norm = False # bool

# monteCarlo_uniform and monteCarlo-coneLocked settings
num_mc = 1000 # number of monte carlo mosaics to generate

Import packages, some additional hardcoded variables (move these to a settings file eventually), helpful type changes, and variable grouping. 

Some of cell would be contained in a separate configuration file, but most of the content from here on in the notebook needs to be contained in an executable.  

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
import os
import h5py
import importlib
import glob
import yaml
import sys

import mosaic_topog.utilities as util
import mosaic_topog.flsyst as flsyst
import mosaic_topog.show as show
import mosaic_topog.calc as calc
import mosaic_topog.singleMosaicProcesses as smp

subject = np.asarray(subject)
angle = np.asarray(angle)
eccentricity = np.asarray(eccentricity)

single_mosaic_savefl_config = 'savefl_config.yml'

coord_fltype = '.csv'
img_fltype = '.png'
filetypes = [coord_fltype, img_fltype]

conetype_sizes = [10, 10, 10, 30]


bin_width = 10

categories = [subject, angle, eccentricity]


# set up for "user_param," a dictionary that is passed to the process manager that
# stores all the variables determined from the user's inputs
user_param_keys = [
                   'subject',
                   'angle',
                   'eccentricity',
                   'conetype',
                   'coord_unit',
                   'bin_width',
                   'dist_area_norm',
                   'conetype_color',
                   'num_mc'
                   ]

waitlist = [
            'mosaic',
            'coord_fl_name',
            'coord_index',
            'img_fl_name',
            'img_index',
            'save_name',
            'processes'
            ] # make sure that parameters in this list are set to user_parm downstream

user_param = dict.fromkeys(user_param_keys)
# most of these parameters have been set directly by the user, 
# a few need to be interpreted from the filesystem, however. 
for p in user_param:
    user_param[p] = locals()[p]

Function scratch space

In [24]:
importlib.reload(flsyst)
importlib.reload(smp)
importlib.reload(calc)
importlib.reload(show)

<module 'mosaic_topog.show' from 'c:\\users\\schle\\documents\\github\\incubator-2022-geometry-of-color\\mosaic_topog\\mosaic_topog\\show.py'>

Get all possible coordinate filenames based on the user inputs and find the ones that exist in the folder. 

In [25]:
# fl_name is the output variable we care about for now -
#     it's a list of all files found for combinations of the input parameters.

# mosaic is a list of all the classified mosaics the located data come from
# for instance a single mosaic may be mapped to mosaic coordinate files for 
# L, M, S, 'all', and 'unclassed' cone sub-mosaics 

# index is a dictionary for indexing through the list flnames according to:
#     inputs to single_mosaic_processes - subject, angle, eccentricity, conetype_ids 
#     determined by single_mosaic_processes - 

mosaic, coord_fl_name, coord_index = flsyst.getFilesByDataGroup(data_path, user_param, '.csv')
img_mosaic, img_fl_name, img_index = flsyst.getFilesByDataGroup(data_path, user_param, '.png')

# Make list of expected save-file names for this data
save_name = flsyst.saveNameFromLoadPath(coord_fl_name,save_path)


found 120 files

found 24 files



Load in the configuration file that determines the Single_mosaic_processes save file structure

In [26]:
sav_cfg = flsyst.readYaml(single_mosaic_savefl_config)
for key in sav_cfg.keys():
    print(key)
    print(sav_cfg[key])
    print('')

mandatory_proc
['meta', 'input_data']

order_optional_proc
['intracone_dist', 'monteCarlo_uniform', 'monteCarlo_uniform_intracone_dist', 'monteCarlo_coneLocked', 'monteCarlo_coneLocked_intracone_dist', 'norm_by_MCU_mean']

meta
{'process': 'meta_process', 'variables': ['mosaic', 'subject', 'angle', 'eccentricity', 'conetype']}

input_data
{'process': 'input_data_process', 'variables': ['cone_img', 'cone_coord', 'coord_unit', 'conetype_color', 'bin_width', 'dist_area_norm', 'num_mc']}

intracone_dist
{'process': 'intracone_dist_process', 'variables': ['dist', 'mean_nearest', 'std_nearest', 'hist', 'bin_edge', 'annulus_area']}

monteCarlo_uniform
{'process': 'monteCarlo_uniform_process', 'variables': ['coord']}

monteCarlo_uniform_intracone_dist
{'process': 'monteCarlo_uniform_intracone_dist_process', 'dependency': None, 'variables': ['dist', 'mean_nearest', 'std_nearest', 'hist', 'bin_edge', 'annulus_area', 'mean_hist', 'std_hist']}

monteCarlo_coneLocked
{'process': 'monteCarlo_coneLoc

If we want to run all the data, start processes.  Otherwise, read from the save folder to determine which data needs to be run through which processes -- then start processes. 

In [27]:
if data_to_run == 'all':
    processes = {}
    for proc in proc_to_run:
        processes[proc] = np.arange(0,len(coord_fl_name)).tolist()
elif data_to_run == 'absent_only':
    processes = flsyst.getProcessesToRun(save_name, save_path, proc_to_run, sav_cfg)


In [28]:
print(processes)

for p in waitlist:
    user_param[p] = vars()[p]
    
# for p in user_param:
#     print(p)
#     print(user_param[p])
#     print('')

{'norm_by_MCU_mean': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]}


In [ ]:
smp.runSingleMosaicProcess(user_param, sav_cfg)